# HPE Container Platform API series  - Lab 3
## Launching cloud-native stateless applications and non-cloud-native stateful applications, programmatically through REST API calls, as a Kubernetes Tenant user.


**Requirements:**
- HPE Container Platform deployment
- IP address of FQDN of the HPE Container Platform's controller host
- a Kubernetes tenant user account   

**Utilities:**   
- cURL  
- Jupyter Notebook server with bash kernel installed
- kubectl, kubectl hpecp plug-in

**Definitions:**
- *HPE Container Platform* is an enterprise-grade container platform designed to deploy both cloud-native and non-cloud-native applications whether on-premises, at the edge, in multiple public clouds, or in a hybrid model. This makes the HPE Container Platform ideal for helping application developers and data scientists accelerate their application development and deployment on **containers**, on-demand through a self-service portal and a RESTful API that surfaces programmable access. To learn more about HPE Container Platform visit the [HPE DEV portal](https://developer.hpe.com/platform/hpe-container-platform/home) and check out the blog articles.

- *tenant:* A tenant is a group of users created by the platform administrator. A tenant can represent for example, an office location, a business unit, an organization, a project, an application. A tenant is allocated a set of resources (CPU/GPU, RAM, Storage, App Store images, Kubernetes cluster) by the platform administrator. All the resources used by a tenant are not shared with other tenants. A tenant user is granted the role of member or admin for the tenant.


HPE Container Platform provides two types of tenants:  
* Big data, AI/ML tenants that exist within the context of HPE Container Platform. This type of tenants are also known as **EPIC** (Elastic Private Instant Cluster) tenants.
* Kubernetes tenants that exist within the context of one or more Kubernetes clusters managed by HPE Container Platform.
  
Here, in this lab part, I will cover how **Kubernetes tenant** users can deploy, using REST API, both cloud-native stateless, microservices applications, and non-cloud-native distributed stateful AI/analytics **KubeDirector** applications on **Kubernetes** cluster managed by HPE Container Platform. Tenant users will then use kubectl to interact directly with the Kubernetes cluster in the context of their tenant.

KubeDirector, also known as Kubernetes Director, is a key component of HPE Container Platform. KubeDirector is an open source project initiated by HPE (BlueData) that enables the running of non-cloud-native stateful analytics workloads on Kubernetes. 

## The HPE Container Platform API Reference
The HPE Container Platform REST API allows you to achieve multiple actions programmatically, from performing administrative tasks, deploying cloud-native and non-cloud-native applications to scoring trained Machine Learning models.

Before you can call the HPE Container Platform API, you need to know what calls can be placed. The REST API reference documentation describes each object type, along with the supported operations, request input parameters, response model and response codes. 
To access the REST API reference documentation, obtain the IP address or hostname of the current active HPE Container Platform controller host from the administrator of the platform. Then in a web browser, navigate to the following URL:

``` 
http(s)://[Controller-IP-address-name]/apidocs
```

All the REST API calls are in the form: 
``` 
An HTTP VERB such as (GET , POST, DELETE, PUT, PATCH, UPDATE),  
A target API object: http(s)://[Controller-IP-address]:8080/api/v2/[object]
```

## Session Authentication in a multi-tenant environment
With the exception of some API calls, most of the REST API calls you can do against the HPE Container Platform API requires authentication. HPE Container Platform uses a *‘session location’* to use as operation context. In a multi-tenant environment, you request an authentication session location by issuing an authentication request in the following form:
* Call the API to request a new login session, providing username/password credentials as well as the Tenant name in the JSON body.  The user must be a valid tenant user credentials with a role (member or admin) in the requested tenant. 
* Extract the resource path of the created *session location* object from the JSON response header,
* For each subsequent call, set a new HTTP Header with its key set to *X-BDS-SESSION* and its value set to the session location value and used as the *working tenant* context. 

``` 
Note: the session location will expire after 24 hours. 
```   

If you are not already familiar with REST API calls, I encourage you to check-out the [Understanding API basics and the value they provide](https://developer.hpe.com/blog/understanding-api-basics-and-the-value-they-provide) blog on HPE Developer Community portal. It explains you REST API concepts such as HTTP verbs you call against the API, the headers, and payloads used when making API calls.  

**cURL:** You will use cURL to make API requests. Information on cURL can be found [here](https://curl.haxx.se/)

#### Initialize the environment.

**IMPORTANT: Before running the next cells, please make sure to adjust the environment variables below according to your Student ID, tenant username and password.**

In [1]:
#
# environment variables to be adjusted by the student
#
student="student80" # your Jupyter Notebook student Identifier (i.e.: student<xx>)
username="student80" # your tenant login credentials - username and password (it matches your Notebook Student account)
password="stuASP2020"
#
# fixed environment variables setup by the HPE CP administrator
#
controller_endpoint="controller.hpedevlab.net:8080"
controller_host="controller.hpedevlab.net"
tenantname="K8sHackTenant"
k8sClusterId="1"  #this is the K8s Cluster Id provided by the HPE CP admisnistrator and assigned to your K8s tenant.
helloWorldApp="hello-world-app.yaml" # the application manifest you will deploy in this lab
tensorFlowApp="tensorflow-notebook-config-cluster.yaml" # the kubedirector application cluster configuration

#### Authenticate as Tenant user in the specified tenant:

In [2]:
sessionlocation=$(curl -k -i -s --request POST "https://${controller_endpoint}/api/v2/session" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' \
--data-raw '{
"name": "'"$username"'",
"password": "'"$password"'",
"tenant_name": "'"$tenantname"'"
}' | grep Location | awk '{print $2}' | tr -d '\r') #we remove any cr that might exist
echo "This is your session location: " $sessionlocation
SessionId=$(echo $sessionlocation | cut -d'/' -f 5) # extract sessionId for later, for logout
echo "This is your session_Id:" $SessionId

This is your session location:  /api/v2/session/61c07af7-95b7-43e2-9840-173cf7b92a15
This is your session_Id: 61c07af7-95b7-43e2-9840-173cf7b92a15


#### Make a quick check to ensure you can make REST API calls within your tenant working context:
Here you will fetch information about your session you have just established.

In [3]:
curl -k -s --request GET "https://${controller_endpoint}/api/v2/session/${SessionId}" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' | jq  #using jq to pretty print the JSON reponse of the API call 

{
  "_links": {
    "self": {
      "href": "/api/v2/session/61c07af7-95b7-43e2-9840-173cf7b92a15"
    },
    "all_sessions": {
      "href": "/api/v2/session"
    }
  },
  "user": "/api/v1/user/803",
  "user_name": "student80",
  "tenant": "/api/v1/tenant/5",
  "tenant_name": "K8sHackTenant",
  "role": "/api/v1/role/3",
  "role_name": "Member",
  "expiry": "2020-4-10 20:11:51",
  "expiry_time": 1586542311,
  "is_site_admin_view": false,
  "is_cluster_superuser": false
}


## Deploying a simple cloud-native stateless application

#### -1- Deploy Hello World containerized application
You will deploy a simple Hello World application on the Kubernetes cluster made available to your tenant. The REST API (POST) call requires the **kubectl** operation type (create or apply) and the application manifest (YAML file that describes the attributes of the application) in a base64 encoded form.  

As you are all sharing the same tenant context and Kubernetes cluster resources, let's make sure your application deployment name will be unique among the tenant users. Here we replace the string "example" with your "username".

In [4]:
sed -i "s/example/${username}/g" $helloWorldApp
cat $helloWorldApp

apiVersion: apps/v1
kind: Deployment
metadata:
  name: hello-world-student80
spec:
  selector:
    matchLabels:
      run: load-balancer-student80
  replicas: 2
  template:
    metadata:
      labels:
        run: load-balancer-student80
    spec:
      containers:
        - name: hello-world-student80
          image: gcr.io/google-samples/node-hello:1.0
          ports:
            - containerPort: 8080
              protocol: TCP
---
apiVersion: v1
kind: Service
metadata:
  name: hello-world-service-student80
spec:
  selector:
    run: load-balancer-student80
  ports:
  - name: http-hello
    protocol: TCP
    port: 8080
    targetPort: 8080
  type: NodePort


In [5]:
#encode the application description file in base64
myapp=$(base64 $helloWorldApp)
#echo $myapp

In [6]:
curl -k -s --request POST "https://${controller_endpoint}/api/v2/k8scluster/${k8sClusterId}/kubectl" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' \
--data-raw '{
"data": "'"$myapp"'",
"op": "apply"
}'

deployment.apps/hello-world-student80 created
service/hello-world-service-student80 created

After a minute or so, you should get the message *Deployment and service created* on the Kubernetes cluster.   

You will now directly interact with the Kubernetes cluster using **kubectl** (the command line configuration tool for Kubernetes).  

In the next steps, you will use kubectl commands in the context of your tenant user account and get the service endpoints of your application to connect to it.

#### -2- Establish the kubectl authenticated context 

The kubectl hpecp plug-in is an extension of the standard kubectl command line. The plugin handles HPECP login and session token management. It is used to establish **HPECP-authenticated** kubectl requests to HPECP-managed Kubernetes services.   

Let's get the kubeconfig file for your working tenant context. The kubeconfig file contains kubernetes context that tenant users can interact with, based on their assigned role (tenant admin or tenant member).

In [7]:
#kubectl version --client
#kubectl hpecp version
kubectl hpecp refresh --insecure-skip-tls-verify ${controller_host} --hpecp-user="${username}" --hpecp-pass="${password}"



Retrieved new Kube Config from HPECP server at controller.hpedevlab.net:8080.
The KUBECONFIG environment variable HAS NOT been set.
Your current session WILL NOT have the new configuration.
To persist these changes by loading all current Kube Config
values into your default Kube Config file, run the
following command:

    KUBECONFIG="/home/student80/.kube/.hpecp/controller.hpedevlab.net/config:/home/student80/.kube/config-backup" kubectl config view --raw > /home/student80/.kube/config

To persist these changes by changing your local KUBECONFIG
environment variable, run the following command:

    export KUBECONFIG="/home/student80/.kube/.hpecp/controller.hpedevlab.net/config"

CAUTION - both of these commands will OVERWRITE your current
Kube Config settings. This is probably what you want, but
to confirm that this command will not break your system,
run the following command to view the resulting Kube
Config file:

    KUBECONFIG="/home/student80/.kube/.hpecp/controller.hpedevlab.n

In [8]:
#define the Kubeconfig file as a shell environment variable
export KUBECONFIG="/home/${student}/.kube/.hpecp/${controller_host}/config"
echo $KUBECONFIG

/home/student80/.kube/.hpecp/controller.hpedevlab.net/config


Check your working tenant context

In [9]:
kubectl config view

apiVersion: v1
clusters:
- cluster:
    insecure-skip-tls-verify: true
    server: https://gateway1.etc.fr.comm.hpecorp.net:9500
  name: k8scluster1
contexts:
- context:
    cluster: k8scluster1
    namespace: k8shacktenant
    user: HPECP-student80
  name: k8scluster1-K8sHackTenant-student80
current-context: k8scluster1-K8sHackTenant-student80
kind: Config
preferences: {}
users:
- name: HPECP-student80
  user:
    exec:
      apiVersion: client.authentication.k8s.io/v1beta1
      args:
      - hpecp
      - authenticate
      - gateway1.etc.fr.comm.hpecorp.net:8080
      - --hpecp-user=student80
      - --hpecp-token=/api/v2/session/ea797839-2194-4a00-9b6d-59d6530961f5
      - --hpecp-token-expiry=1586542343
      - --force-reauth=false
      - --insecure-skip-tls-verify=true
      command: kubectl
      env: null


#### -3- Get the POD and Services for your deployed application:

In [10]:
kubectl get pod,service | grep ${username}

pod/hello-world-student80-57cbc4fb7c-ft6d5   1/1     Running   0          24s
pod/hello-world-student80-57cbc4fb7c-s4gx4   1/1     Running   0          24s
service/hello-world-service-student80   NodePort   10.96.176.84   <none>        8080:31361/TCP   24s


In [11]:
kubectl describe service hello-world-service-${username}

Name:                     hello-world-service-student80
Namespace:                k8shacktenant
Labels:                   hpecp.hpe.com/hpecp-internal-gateway=true
Annotations:              hpecp-internal-gateway/8080: gateway1.etc.fr.comm.hpecorp.net:10002
                          kubectl.kubernetes.io/last-applied-configuration:
                            {"apiVersion":"v1","kind":"Service","metadata":{"annotations":{},"name":"hello-world-service-student80","namespace":"k8shacktenant"},"spec...
Selector:                 run=load-balancer-student80
Type:                     NodePort
IP:                       10.96.176.84
Port:                     http-hello  8080/TCP
TargetPort:               8080/TCP
NodePort:                 http-hello  31361/TCP
Endpoints:                10.192.0.185:8080,10.192.1.57:8080
Session Affinity:         None
External Traffic Policy:  Cluster
Events:
  Type    Reason  Age   From         Message
  ----    ------  ----  ----         -------
  Normal  HpeC

#### -4- Get the service endpoints
HPE Container Platform automatically maps the NodePort Service endpoint to the HPE Container Platform gateway (proxy) host.
Access to application services running in containers is proxied via the gateway host and a port number greater than 10000.

In [12]:
myservice="hello-world-service-"${username}
appURL=$(kubectl describe service/"${myservice}" | grep gateway1 | awk '{print $3}')
myapp_endpoint="https://${appURL}"
echo "Your application service endpoint is: "$myapp_endpoint

Your application service endpoint is: https://gateway1.etc.fr.comm.hpecorp.net:10002


#### -5- Check your application is responding:

In [14]:
curl -k -s "${myapp_endpoint}"

Hello Kubernetes!

#### -6- Delete your stateless application deployment
Delete your application deployment and services using the REST API (POST) call below. The REST API call requires the **kubectl** operation type (delete) and the application manifest in a base64 encoded form.
After a minute or so, you should get the message: deployment deleted and service deleted.

In [15]:
curl -k -s --request POST "https://${controller_endpoint}/api/v2/k8scluster/${k8sClusterId}/kubectl" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' \
--data-raw '{
"data": "'"$myapp"'",
"op": "delete"
}'

deployment.apps "hello-world-student80" deleted
service "hello-world-service-student80" deleted

## Deploying a KubeDirector Application

HPE has been working within the open source Kubernetes community to add capabilities that enable the running of **stateful** analytics workloads on Kubernetes.  The open source project is known as **Kubernetes Director** or **KubeDirector** for short.   
With KubeDirector, enterprises can deploy all their enterprise applications on a common Kubernetes framework.
KubeDirector is deployed as a custom controller (aka an operator) by default on every Kubernetes cluster managed by the HPE Container Platform.

For more information about KubeDirector, check out the HPE DEV portal [here](https://developer.hpe.com/platform/hpe-container-platform/home).

#### -1- List the installed KubeDirector Application
In the current release of HPE Container Platform, the Kubernetes cluster managed by HPE Controller platform has three (3) pre-configured KubeDirector application types installed out of the box. A tenant user who would access the HPE Container Platform portal UI could see it:

<img src="Pictures/HPECP-KubeDirectorApp-GUI.png" height="800" width="600" align="left">

You can also get the list of KubeDirector Applications using kubectl command:

In [16]:
kubectl get kubedirectorapp

NAME                  AGE
centos7x              16d
ml-jupyter-notebook   16d
spark221e2            16d


A KubeDirectorApp describes an application type, its images, service roles and endpoints, and if appropriate, the root file system directories (e.g.: /etc, /bin, /opt, /var, /usr) of the containers to persist beyond the life span of the container. This means stateful applications that write data to their root file systems can now successfully run on Kubernetes. Let's inspect a couple of KubeDirector application types: 

In [17]:
kubectl describe kubedirectorapp ml-jupyter-notebook 

Name:         ml-jupyter-notebook
Namespace:    k8shacktenant
Labels:       <none>
Annotations:  token:
                execute this command to get the authentication token 'kubectl exec <pod-name> jupyter notebook list' and use this token in Jupyter noteboo...
API Version:  kubedirector.hpe.com/v1beta1
Kind:         KubeDirectorApp
Metadata:
  Creation Timestamp:  2020-03-24T08:25:15Z
  Generation:          2
  Resource Version:    2874116
  Self Link:           /apis/kubedirector.hpe.com/v1beta1/namespaces/k8shacktenant/kubedirectorapps/ml-jupyter-notebook
  UID:                 cb7a6132-f321-45d6-81f5-635f0b79e61e
Spec:
  Config:
    Role Services:
      Role ID:  controller
      Service I Ds:
        jupyter-nb
    Selected Roles:
      controller
  Config Schema Version:   7
  Default Config Package:  <nil>
  Distro ID:               bluedata/tensorflow
  Label:
    Description:  TensorFlow GPU with Jupyter notebook
    Name:         TensorFlow + Jupyter
  Roles:
    Cardinality:

In [18]:
kubectl describe kubedirectorapp spark221e2

Name:         spark221e2
Namespace:    k8shacktenant
Labels:       <none>
Annotations:  <none>
API Version:  kubedirector.hpe.com/v1beta1
Kind:         KubeDirectorApp
Metadata:
  Creation Timestamp:  2020-03-24T08:25:15Z
  Generation:          1
  Resource Version:    981021
  Self Link:           /apis/kubedirector.hpe.com/v1beta1/namespaces/k8shacktenant/kubedirectorapps/spark221e2
  UID:                 753618be-1edf-470c-bff5-385d0e76fafe
Spec:
  Config:
    Role Services:
      Role ID:  controller
      Service I Ds:
        ssh
        spark
        spark-master
        spark-worker
      Role ID:  worker
      Service I Ds:
        ssh
        spark-worker
      Role ID:  jupyter
      Service I Ds:
        ssh
        jupyter-nb
    Selected Roles:
      controller
      worker
      jupyter
  Config Schema Version:  7
  Distro ID:              bluedata/spark221e2
  Label:
    Description:  Spark 2.2.1 with Jupyter notebook
    Name:         Spark 2.2.1 + Jupyter
  Roles:
   

#### -2- Deploy the TensorFlow with Jupyter Notebook stateful application
A configuration manifest YAML file is used to create KubeDirector virtual clusters that instantiate the defined KubeDirector App type.

We need to make sure the instance of the kubedirector application is unique among your tenant. We also need to convert the application configuration manifest YAML file in a base64 encoded form.

In [19]:
sed -i "s/example/${username}/g" $tensorFlowApp
cat $tensorFlowApp

---
apiVersion: "kubedirector.hpe.com/v1beta1"
kind: "KubeDirectorCluster"
metadata: 
  name: "ml-jupyter-notebook-student80"
spec: 
  app: "ml-jupyter-notebook"
  appCatalog: "local"
  roles: 
    - 
      id: "controller"
      members: 1
      resources: 
        requests: 
          memory: "4Gi"
          cpu: "2"
          nvidia.com/gpu: "0"
        limits: 
          memory: "4Gi"
          cpu: "2"
          nvidia.com/gpu: "0"


The configuration manifest specifies the application instance name, the KubeDirector App type, the application service roles and their number of nodes and compute size. Let's deploy it now using the REST API call below. The REST API call requires the **kubectl** operation type (create or apply) and the application manifest in a base64 encoded form.

In [20]:
mytfapp=$(base64 $tensorFlowApp)
#echo $mytfapp
curl -k -s --request POST "https://${controller_endpoint}/api/v2/k8scluster/${k8sClusterId}/kubectl" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' \
--data-raw '{
"data": "'"$mytfapp"'",
"op": "apply"
}'

kubedirectorcluster.kubedirector.hpe.com/ml-jupyter-notebook-student80 created

After a minute or so, you should get the message *kubedirectorcluster/Your-instance-name created*.  

Once you get this message, go to the next step.

#### -3- Inspect the deployed KubeDirector application virtual cluster 
Your application virtual cluster will be represented in the Kubernetes cluster by a resource of type **KubeDirectorCluster**, with the name that was indicated inside the YAML file used to create it. **A kubeDirectorCluster resource is an instantiation of a KubeDirector App**.

In [21]:
clusterName="ml-jupyter-notebook-${username}"
kubectl get kubedirectorcluster $clusterName

NAME                            AGE
ml-jupyter-notebook-student80   7s


After creating the instance of the KubeDirector App, you can use kubectl command below to observe its status and the Kubernetes resources that make up the application virtual cluster (statefulsets, pods, services, persistent volume claim if any) and any events logged against it.

The virtual cluster status indicates its overall "state" (top-level property of the status object). It should have a value of **"configured"**. The first time a virtual cluster of a given KubeDirector App type is created, it may take some minutes to reach "configured" state, as the relevant Docker image must be downloaded and imported.

In [22]:
kubectl describe kubedirectorcluster $clusterName

Name:         ml-jupyter-notebook-student80
Namespace:    k8shacktenant
Labels:       <none>
Annotations:  kubectl.kubernetes.io/last-applied-configuration:
                {"apiVersion":"kubedirector.hpe.com/v1beta1","kind":"KubeDirectorCluster","metadata":{"annotations":{},"name":"ml-jupyter-notebook-student...
API Version:  kubedirector.hpe.com/v1beta1
Kind:         KubeDirectorCluster
Metadata:
  Creation Timestamp:  2020-04-09T18:14:49Z
  Finalizers:
    kubedirector.hpe.com/cleanup
  Generation:        1
  Resource Version:  4369742
  Self Link:         /apis/kubedirector.hpe.com/v1beta1/namespaces/k8shacktenant/kubedirectorclusters/ml-jupyter-notebook-student80
  UID:               e6570af8-b812-46d2-8de8-f2492a04f4b2
Spec:
  App:          ml-jupyter-notebook
  App Catalog:  local
  Roles:
    Id:       controller
    Members:  1
    Resources:
      Limits:
        Cpu:             2
        Memory:          4Gi
        nvidia.com/gpu:  0
      Requests:
        Cpu:           

In [23]:
kubectl get all -l kubedirector.hpe.com/kdcluster=$clusterName

NAME               READY   STATUS    RESTARTS   AGE
pod/kdss-fchtn-0   1/1     Running   0          19s

NAME                     TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)          AGE
service/kdhs-hqvzl       ClusterIP   None           <none>        8888/TCP         19s
service/s-kdss-fchtn-0   NodePort    10.96.10.145   <none>        8888:31088/TCP   19s

NAME                          READY   AGE
statefulset.apps/kdss-fchtn   1/1     20s


Your KubeDirector application virtual cluster is made up of a **statefulset**, a **pod** and **services**.   

The NodePort service exposes the application outside the Kubernetes cluster. 
HPE Container Platform automatically maps the NodePort Service endpoint to the HPE Container Platform gateway (proxy) host.

#### -4- Get the application service endpoints
To get a report on all services related to a specific virtual cluster, you can use a form of "kubectl get" that matches against a value of the **kubedirector.hpe.com/kdcluster=YourClusterApplicationName** label.

In [24]:
tfappURL=$(kubectl describe service -l  kubedirector.hpe.com/kdcluster=${clusterName} | grep gateway1 | awk '{print $3}')
mytfapp_endpoint="https://${tfappURL}"
echo "Your application virtual cluster service endpoint is: "$mytfapp_endpoint

Your application virtual cluster service endpoint is: https://gateway1.etc.fr.comm.hpecorp.net:10002


Access to application services running in containers is proxied via the gateway host and a port number greater than 10000.

#### -5- Check the application service is responding

In [25]:
curl -k -L -s -i "${mytfapp_endpoint}" | grep "HTTP/1.1"

HTTP/1.1 302 Found
HTTP/1.1 302 Found
HTTP/1.1 200 OK


#### -6- Fetch the authentication token of TensorFlow Jupyter Notebook 

Execute this command to get the authentication token *'kubectl exec pod-name jupyter notebook list'* and use this token in Jupyter notebook

In [26]:
mytfapp_pod=$(kubectl get pod -l kubedirector.hpe.com/kdcluster=${clusterName} | grep kdss | awk '{print $1}') 
#echo $mytfapp_pod
kubectl exec ${mytfapp_pod} jupyter notebook list

Currently running servers:
http://0.0.0.0:8888/?token=0d5147c7486ee49d6f1c33de79ad7e23ef50ff53552bfd4e :: /tf


#### -7- Connect to your application

You can connect to your tensorflow framework with Jupyter Notebook and start build your ML model (for example a regression model), train the model and make prediction.

Open a new tab in your browser, and connect to the service endpoint: https://77.158.163.130:YourPortNumber?token=YourToken
where 77.158.163.130 is the NAT IP address of the HPE CP proxy gateway, port number is the port you get for your service endpoint, and the token is the authentication token you have just fetched. 

![TensorFlow Jupyter Notebook](Pictures/kubedirector-tf-jupyterNotebook.png)

#### -8- Go through some cleanup

* First delete the kubedirector application cluster instance using the REST API (POST) call below. The REST API call requires the **kubectl** operation type (delete) and the application manifest in a base64 encoded form.
Deleting the KubeDirectorCluster resource will automatically delete all resources (for example the pods, services, statefulset) that compose the KubeDirector application virtual cluster.

In [27]:
curl -k -s --request POST "https://${controller_endpoint}/api/v2/k8scluster/${k8sClusterId}/kubectl" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' \
--data-raw '{
"data": "'"$mytfapp"'",
"op": "delete"
}'

kubedirectorcluster.kubedirector.hpe.com "ml-jupyter-notebook-student80" deleted

* Although session have a time to live (TTL) of 24 hours, it is best practice in REST API programming to cleanup and delete those sessions when done. We can use a DELETE /api/v2/session/SessionId to achieve this.

In [28]:
curl -k -i -s --request DELETE "https://${controller_endpoint}/api/v2/session/${SessionId}" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json'

HTTP/1.1 204 No Content
Access-Control-Allow-Origin: *
Content-Length: 0
Content-Type: text/plain
Date: Thu, 09 Apr 2020 18:25:35 GMT
Server: BlueData EPIC 5.0



The status *204 No Content* means the session object has been deleted.

* Finally, reset your applications YAML files

In [29]:
#reset the application deployment name in the YAML file
sed -i "s/${username}/example/g" $helloWorldApp
sed -i "s/${username}/example/g" $tensorFlowApp
cat $helloWorldApp
cat $tensorFlowApp

apiVersion: apps/v1
kind: Deployment
metadata:
  name: hello-world-example
spec:
  selector:
    matchLabels:
      run: load-balancer-example
  replicas: 2
  template:
    metadata:
      labels:
        run: load-balancer-example
    spec:
      containers:
        - name: hello-world-example
          image: gcr.io/google-samples/node-hello:1.0
          ports:
            - containerPort: 8080
              protocol: TCP
---
apiVersion: v1
kind: Service
metadata:
  name: hello-world-service-example
spec:
  selector:
    run: load-balancer-example
  ports:
  - name: http-hello
    protocol: TCP
    port: 8080
    targetPort: 8080
  type: NodePort
---
apiVersion: "kubedirector.hpe.com/v1beta1"
kind: "KubeDirectorCluster"
metadata: 
  name: "ml-jupyter-notebook-example"
spec: 
  app: "ml-jupyter-notebook"
  appCatalog: "local"
  roles: 
    - 
      id: "controller"
      members: 1
      resources: 
        requests: 
          memory: "4Gi"
          cpu: "2"
          nvidia.com/gp

## Summary

In this tutorial, you learned how **Kubernetes tenant** users can deploy, using REST API, both cloud-native stateless, microservices applications, and non-cloud-native distributed stateful AI/analytics KubeDirector applications on Kubernetes clusters managed by HPE Container Platform. You also used the standard Kubernetes command line (kubectl) as well as the HPE CP plugin to establish HPECP-authenticated kubectl requests in the context of your tenant user account. 